In [1]:
import os
import ast
import pickle
import swifter
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask import compute, delayed
import multiprocessing

from model import DeepMALRawPackets
from sklearn.metrics import confusion_matrix,  ConfusionMatrixDisplay
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.WARN)  # or any {DEBUG, INFO, WARN, ERROR, FATAL}

# Early stopping & checkpoints
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
# # Obtener la ruta de la carpeta donde están los archivos .csv
# carpeta_benign = './Dataset/Benign'
# # Obtener una lista de todos los archivos .csv en la carpeta
# benign_csv_list = [archivo for archivo in os.listdir(carpeta_benign) if archivo.endswith('.csv')]

# # Leer los archivos .csv y concatenarlos en un solo dataframe
# df = pd.concat([pd.read_csv(os.path.join(carpeta_benign, archivo)) for archivo in benign_csv_list])

# #Reiniciar los indices del DataFrame unificado
# df = df.reset_index(drop=True)

In [3]:
# Obtener la ruta de la carpeta donde están los archivos .csv
# carpeta_benign = './Dataset/Benign'

# # Obtener una lista de todos los archivos .csv en la carpeta
# benign_csv_list = [archivo for archivo in os.listdir(carpeta_benign) if archivo.endswith('.csv')]

# # Obtener la ruta de la carpeta donde se guardarán los archivos pickle transformados
# carpeta_transformados = './Dataset/TransformadosBenign'

# # Crear la carpeta si no existe
# if not os.path.exists(carpeta_transformados):
#     os.makedirs(carpeta_transformados)

# # Leer los archivos .csv y aplicar la transformación a cada uno
# # df_list = []
# for archivo in benign_csv_list:
#     df = pd.read_csv(os.path.join(carpeta_benign, archivo))
#     df['udps.n_bytes_per_packet'] = df['udps.n_bytes_per_packet'].swifter.apply(eval)
#     # df_list.append(df)
#     # Guardar el DataFrame transformado como un archivo pickle en la carpeta de transformados
#     nombre_archivo = os.path.splitext(archivo)[0] + '_transformado.pickle'
#     ruta_archivo = os.path.join(carpeta_transformados, nombre_archivo)
#     with open(ruta_archivo, 'wb') as f:
#         pickle.dump(df, f)

# # Concatenar los DataFrames en un solo DataFrame
# df = pd.concat(df_list)

# # Reiniciar los índices del DataFrame unificado
# df = df.reset_index(drop=True)

In [4]:
# Obtener la ruta de la carpeta donde están los archivos .csv
# carpeta_malware = './Dataset/Malware'

# # Obtener una lista de todos los archivos .csv en la carpeta
# malware_csv_list = [archivo for archivo in os.listdir(carpeta_malware) if archivo.endswith('.csv')]

# # Obtener la ruta de la carpeta donde se guardarán los archivos pickle transformados
# carpeta_transformados = './Dataset/TransformadosMalware'

# # Crear la carpeta si no existe
# if not os.path.exists(carpeta_transformados):
#     os.makedirs(carpeta_transformados)

# # Leer los archivos .csv y aplicar la transformación a cada uno
# # df_list = []
# for archivo in malware_csv_list:
#     df_doh = pd.read_csv(os.path.join(carpeta_malware, archivo))
#     df_doh = df_doh.iloc[:5000]
#     df_doh['udps.n_bytes_per_packet'] = df_doh['udps.n_bytes_per_packet'].swifter.apply(eval)
#     # df_list.append(df)
#     # Guardar el DataFrame transformado como un archivo pickle en la carpeta de transformados
#     nombre_archivo = os.path.splitext(archivo)[0] + '_transformado.pickle'
#     ruta_archivo = os.path.join(carpeta_transformados, nombre_archivo)
#     with open(ruta_archivo, 'wb') as f:
#         pickle.dump(df_doh, f)

In [5]:
# Obtener la ruta de la carpeta donde están los archivos .csv
carpeta_benign = './Dataset/Benign'
# Obtener una lista de todos los archivos .csv en la carpeta
benign_csv_list = [archivo for archivo in os.listdir(carpeta_benign) if archivo.endswith('.csv')]

# Leer los archivos .csv y concatenarlos en un solo dataframe
df = pd.concat([pd.read_csv(os.path.join(carpeta_benign, archivo)) for archivo in benign_csv_list])

#Reiniciar los indices del DataFrame unificado
df = df.reset_index(drop=True)

In [6]:
# Obtener la ruta de la carpeta donde están los archivos .csv
carpeta_malware = './Dataset/Malware'
# Obtener una lista de todos los archivos .csv en la carpeta
malware_csv_list = [archivo for archivo in os.listdir(carpeta_malware) if archivo.endswith('.csv')]

# Leer los archivos .csv y concatenarlos en un solo dataframe
df_doh = pd.concat([pd.read_csv(os.path.join(carpeta_malware, archivo)) for archivo in malware_csv_list])

#Reiniciar los indices del DataFrame unificado
df_doh = df_doh.reset_index(drop=True)

In [7]:
###########################
# Importar datos benignos #
# carpeta_transformados_benign = './Dataset/TransformadosBenign'

# # Obtener la lista de archivos pickle en la carpeta
# pickle_files = [f for f in os.listdir(carpeta_transformados_benign) if f.endswith('.pickle')]

# # Crear una lista vacía para almacenar los dataframes cargados
# df_list = []

# # Cargar cada archivo pickle en la lista
# for file in pickle_files:
#     with open(os.path.join(carpeta_transformados_benign, file), 'rb') as f:
#         df = pickle.load(f)
#         df_list.append(df)

# # Combinar los dataframes cargados en un solo dataframe
# df = pd.concat(df_list, ignore_index=True)

In [8]:
###########################
# Importar datos malignos #
# carpeta_transformados_malware = './Dataset/TransformadosMalware'

# # Obtener la lista de archivos pickle en la carpeta
# pickle_files = [f for f in os.listdir(carpeta_transformados_malware) if f.endswith('.pickle')]

# # Crear una lista vacía para almacenar los dataframes cargados
# df_doh_list = []

# # Cargar cada archivo pickle en la lista
# for file in pickle_files:
#     with open(os.path.join(carpeta_transformados_malware, file), 'rb') as f:
#         df_doh = pickle.load(f)
#         df_doh_list.append(df_doh)

# # Combinar los dataframes cargados en un solo dataframe
# df_doh = pd.concat(df_doh_list, ignore_index=True)

In [9]:
print("Benign data shape: " + str(df.shape))
print("Malware data shape: " + str(df_doh.shape))

Benign data shape: (309887, 1)
Malware data shape: (179252, 1)


In [10]:
#print(df['udps.n_bytes_per_packet'].isnull().sum())
#print(df['udps.n_bytes_per_packet'].isna().sum())

#print(df_doh['udps.n_bytes_per_packet'].isnull().sum())
#print(df_doh['udps.n_bytes_per_packet'].isna().sum())

In [11]:
# Crear una lista de índices de filas que contienen un error de clave
error_indices = []
for idx in df_doh.index:
    if (len(df_doh.loc[idx]['udps.n_bytes_per_packet']) > 1000000):
        error_indices.append(idx)

# print(len(error_indices))
# Eliminar las filas que contienen un error de clave
df_doh = df_doh.drop(error_indices)

In [12]:
df_doh = df_doh.reset_index(drop=True)

In [13]:
# Cut data
# df = df.iloc[:186000]
# df_doh = df_doh.iloc[:108000]

In [14]:
# Cut data
df = df.iloc[:int(df.shape[0] * 0.4)]
df_doh = df_doh.iloc[:int(df_doh.shape[0] * 0.32)]

In [15]:
df['udps.n_bytes_per_packet'] = df['udps.n_bytes_per_packet'].swifter.apply(eval)

Pandas Apply:   0%|          | 0/92966 [00:00<?, ?it/s]

In [16]:
df_doh['udps.n_bytes_per_packet'] = df_doh['udps.n_bytes_per_packet'].swifter.apply(eval)

Pandas Apply:   0%|          | 0/71478 [00:00<?, ?it/s]

In [17]:
df['label'] = np.zeros(len(df))
df_doh['label'] = np.ones(len(df_doh))

In [18]:
data, data_doh = df, df_doh
print(data.memory_usage(index=True, deep=True))
print(data_doh.memory_usage(index=True, deep=True))

deep_mal_classifier = DeepMALRawPackets()
print(deep_mal_classifier.model.summary())

Index                          128
udps.n_bytes_per_packet    7063152
label                       743728
dtype: int64
Index                          128
udps.n_bytes_per_packet    9800672
label                       571824
dtype: int64
Model: "DeepMAL-Packets"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1024, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 1020, 32)          192       
                                                                 
 conv1d_1 (Conv1D)           (None, 1016, 64)          10304     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 127, 64)          0         
 )                                                               
                                                                 
 lstm (LSTM) 

In [19]:
# Training
print('[Status] Training model...')
train_data = pd.concat([data.sample(frac=1), data_doh.sample(frac=1)]).sample(frac=1)
train_data = train_data.explode('udps.n_bytes_per_packet', ignore_index=True)
print(train_data.memory_usage(index=True, deep=True))
x_train = list(train_data['udps.n_bytes_per_packet'])
y_train = list(train_data['label'])
print('Amount of instances', len(train_data.index))
print('Amount of positive instances:', len(train_data[train_data['label']==1].index))

earlyStop = EarlyStopping(monitor='binary_accuracy', patience=7)
model_checkpoint = ModelCheckpoint(filepath='deepmal-{epoch:02d}-{binary_accuracy:.2f}.h5', monitor='binary_accuracy', verbose=0, save_best_only=True, save_freq="epoch")

epochs = 5
batch_size = 32
deep_mal_classifier.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, class_weight={0: 0.5, 1: 0.5}, verbose=2, callbacks=[earlyStop, model_checkpoint]) # {0: 0.012, 1: 0.988}

[Status] Training model...
Index                             128
udps.n_bytes_per_packet    8937598400
label                         8677280
dtype: int64
Amount of instances 1084660
Amount of positive instances: 759562
Epoch 1/10
33896/33896 - 21483s - loss: 0.1423 - binary_accuracy: 0.8067 - recall: 0.9970 - precision: 0.7850 - 21483s/epoch - 634ms/step
Epoch 2/10
33896/33896 - 23002s - loss: 0.1436 - binary_accuracy: 0.8068 - recall: 0.9972 - precision: 0.7850 - 23002s/epoch - 679ms/step
Epoch 3/10
33896/33896 - 26089s - loss: 0.1409 - binary_accuracy: 0.8091 - recall: 0.9977 - precision: 0.7868 - 26089s/epoch - 770ms/step
Epoch 4/10
33896/33896 - 26059s - loss: 0.1453 - binary_accuracy: 0.8098 - recall: 0.9977 - precision: 0.7874 - 26059s/epoch - 769ms/step
Epoch 5/10
33896/33896 - 25572s - loss: 0.1495 - binary_accuracy: 0.8088 - recall: 0.9974 - precision: 0.7867 - 25572s/epoch - 754ms/step
Epoch 6/10
33896/33896 - 25468s - loss: 0.1467 - binary_accuracy: 0.8036 - recall: 0.9962 -

In [20]:
# predict
print('[Status] Predicting...')
test_data  = pd.concat([data.sample(frac=0.5), data_doh.sample(frac=0.5)]).sample(frac=1)
test_data = test_data.explode('udps.n_bytes_per_packet', ignore_index=True)
x_test = list(test_data['udps.n_bytes_per_packet'])
y_test = list(test_data['label'])
predictions = deep_mal_classifier.model.predict(x_test)
predictions[predictions>0.5] = 1
predictions[predictions<=0.5] = 0
print(sum(predictions), len(predictions))
print(confusion_matrix(y_test, predictions))

[Status] Predicting...
16944/16944 [==============================] - 5827s 344ms/step
[480657.] 542180
[[ 61137 101315]
 [   386 379342]]


In [21]:
# Exportar el modelo entrenado
deep_mal_classifier.model.save("deep_mal_v8.h5")